In [1]:
from datetime import datetime, timedelta
import re
import sqlite3
import time
import os
import pandas as pd
pd.set_option('display.max_columns', None)
from bs4 import BeautifulSoup as bs4
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import shutil
from sqlalchemy import MetaData, text, Column, Integer, String, ForeignKey, Table, create_engine, Float, Boolean, DateTime

In [2]:
from scraper import Scraper

In [3]:
s = Scraper()

Change the system name (atc, steamer, zips, thebat, thebatx, fangraphsdc, oopsy) and stats type (either bat or pit)

In [9]:
s.get_fangraphs_projections('oopsy',stats_type='bat')

'2025-oopsy-proj-h.csv saved in C:\\Users\\pddnh\\Documents\\GitHub\\xdl\\data'

### Manual approach

In [7]:
def login(path=None):
    service = Service(executable_path=path)
    driver = webdriver.Chrome(service=service)
    driver.get('https://blogs.fangraphs.com/wp-login.php?redirect_to=https://www.fangraphs.com/projections?pos=all&stats=bat&type=steamer')
    driver.implicitly_wait(5)
    login_form = driver.find_element(By.ID, 'loginform')
    login = driver.find_element(By.ID, 'user_login')
    login.send_keys('Haueter19')
    time.sleep(1.2)
    pw = driver.find_element(By.ID, 'user_pass')
    pw.send_keys('Segneri9@')
    time.sleep(1.4)
    login_form.submit()
    return driver

In [5]:
path = r"C:\ProgramData\Anaconda3\WebDriver\bin\chromedriver.exe"

In [8]:
driver = login(path)

In [9]:
driver.get('https://www.fangraphs.com/projections?pos=all&type=steamer&statgroup=fantasy&fantasypreset=dashboard')

In [10]:
# Download to Downloads path
driver.find_element(By.CLASS_NAME, 'data-export').click()

In [11]:
download_path = r'C:\Users\pddnh\Downloads'
downloaded_fangraphs_filename = 'fangraphs-leaderboard-projections.csv'
dest_path = r"C:\Users\pddnh\Documents\GitHub\xdl\data"
proj = 'steamer'
proj_type='h'
filename = f"{datetime.now().year}-{proj}-proj-{proj_type}.csv"

In [12]:
# Rename downloaded file appropriately
os.rename(os.path.join(download_path,downloaded_fangraphs_filename), os.path.join(download_path,filename))

# Move file to data/ folder
shutil.move(os.path.join(download_path,filename), os.path.join(dest_path,filename))

'C:\\Users\\pddnh\\Documents\\GitHub\\xdl\\data\\2025-steamer-proj-h.csv'

In [21]:
driver.get('https://www.fangraphs.com/projections?pos=&type=zips&statgroup=fantasy&fantasypreset=roto5x5&stats=pit')

In [22]:
driver.find_element(By.CLASS_NAME, 'data-export').click()

In [23]:
download_path = r'C:\Users\pddnh\Downloads'
downloaded_fangraphs_filename = 'fangraphs-leaderboard-projections.csv'
destination_path = r"C:\Users\pddnh\Documents\GitHub\xdl\data"
proj = 'zips'
proj_type='p'
filename = f"{datetime.now().year}-{proj}-proj-{proj_type}.csv"
filename

'2025-zips-proj-p.csv'

In [16]:
# Rename downloaded file appropriately
os.rename(os.path.join(download_path,downloaded_fangraphs_filename), os.path.join(download_path,filename))

# Move file to data/ folder
shutil.move(os.path.join(download_path,filename), os.path.join(dest_path,filename))

'C:\\Users\\pddnh\\Documents\\GitHub\\xdl\\data\\2025-steamer-proj-p.csv'

In [17]:
fg_types = ['zips', 'zipsdc', 'steamer', 'fangraphsdc', 'atc', 'thebat', 'thebatx']
stat_types = ['bat', 'pit']
statgroup = 'fantasy'
fantasypreset = 'roto5x5'

In [18]:
df = pd.read_csv(os.path.join(dest_path,filename))

In [19]:
df

,Name,Team,W,L,QS,ERA,G,GS,SV,HLD,BS,IP,TBF,H,R,ER,HR,BB,IBB,HBP,SO,K/9,BB/9,K/BB,HR/9,K%,BB%,K-BB%,AVG,WHIP,BABIP,LOB%,GB%,HR/FB,FIP,WAR,RA9-WAR,ADP,InterSD,InterSK,IntraSD,Vol,Skew,Dim,FPTS,FPTS/IP,SPTS,SPTS/IP,P10,P20,P30,P40,P50,P60,P70,P80,P90,TT10,TT20,TT30,TT40,TT50,TT60,TT70,TT80,TT90,NameASCII,PlayerId,MLBAMID
0,Paul Skenes,PIT,13.1606,8.6755,21.1085,2.79917,30.78,30.78,0.0,0.0000,NaN,188.104,759.711,147.4100,64.6952,58.5040,15.50820,55.0433,0.4368,7.5295,241.7550,11.56700,2.63359,4.39209,0.742,0.318220,0.072453,0.245767,0.211450,1.076280,0.299862,0.762181,0.470836,NaN,2.66577,5.802600,5.770480,11.240000,NaN,NaN,NaN,NaN,NaN,NaN,1107.2,5.889362,1027,5.462766,4.21,3.74,3.44,3.20,2.99,2.79,2.60,2.38,2.12,3.71,3.48,3.31,3.18,3.06,2.95,2.83,2.69,2.52,Paul Skenes,33677,694973
1,Jacob deGrom,TEX,11.1856,6.8150,16.5718,2.85724,25.92,25.92,0.0,0.0000,NaN,152.018,602.049,115.6140,52.3611,48.2612,17.16110,30.0814,0.3016,4.9242,205.9420,12.19260,1.78093,6.84617,1.016,0.342069,0.049965,0.292104,0.203889,0.958411,0.286250,0.767111,0.383583,NaN,2.61540,5.131270,4.357470,40.029999,NaN,NaN,NaN,NaN,NaN,NaN,921.1,6.059868,846,5.565789,4.32,3.80,3.46,3.20,2.97,2.76,2.55,2.33,2.05,3.75,3.50,3.33,3.19,3.06,2.94,2.82,2.68,2.50,Jacob deGrom,10954,594798
2,Tarik Skubal,DET,13.3054,9.4571,21.6253,2.92117,32.00,32.00,0.0,0.0000,NaN,196.234,787.456,160.2090,69.9435,63.6926,20.03770,45.0007,0.3016,7.9903,228.6170,10.48520,2.06389,5.08029,0.919,0.290323,0.057147,0.233176,0.218131,1.045740,0.288532,0.763539,0.435306,NaN,2.97400,4.989030,5.114100,13.050000,NaN,NaN,NaN,NaN,NaN,NaN,1087.4,5.547959,1019,5.198980,4.26,3.83,3.54,3.31,3.11,2.92,2.74,2.53,2.27,3.72,3.53,3.39,3.28,3.18,3.09,2.99,2.88,2.73,Tarik Skubal,22267,669373
3,Garrett Crochet,BOS,12.3448,8.4609,17.3777,3.03826,30.78,30.78,0.0,0.0000,NaN,170.138,684.110,133.0210,62.6500,57.4360,16.61680,51.7078,0.3927,6.5394,220.7620,11.67790,2.73525,4.26941,0.879,0.322699,0.075584,0.247115,0.212541,1.085760,0.299638,0.756069,0.439859,NaN,2.86812,4.809880,4.878950,23.000000,NaN,NaN,NaN,NaN,NaN,NaN,968.1,5.694706,894,5.258824,4.49,4.00,3.68,3.42,3.20,2.99,2.78,2.56,2.28,3.93,3.69,3.53,3.40,3.28,3.17,3.05,2.92,2.74,Garrett Crochet,27463,676979
4,Zack Wheeler,PHI,13.4865,9.4027,20.5873,3.32157,32.00,32.00,0.0,0.0000,NaN,198.518,809.947,171.0910,79.8276,73.2657,22.80750,50.9295,0.8180,8.6624,212.4860,9.63327,2.30894,4.17217,1.034,0.262346,0.062880,0.199466,0.228014,1.118390,0.287896,0.748030,0.425639,NaN,3.41986,4.625790,4.752140,20.920000,NaN,NaN,NaN,NaN,NaN,NaN,989.1,4.970352,940,4.723618,4.71,4.27,3.97,3.74,3.53,3.33,3.13,2.92,2.64,4.12,3.93,3.80,3.70,3.60,3.51,3.41,3.30,3.15,Zack Wheeler,10310,554430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5209,Amos Willingham,ATL,2.2678,2.2348,0.0000,4.54706,46.00,0.00,0.0,0.6209,NaN,46.000,200.786,46.9733,24.9158,23.2405,6.47067,17.4631,1.2461,2.1147,39.7114,7.76962,3.41669,2.27402,1.266,0.197780,0.086974,0.110806,0.259223,1.400790,0.299961,0.713659,0.407898,NaN,4.54538,-0.230152,-0.072984,999.000000,NaN,NaN,NaN,NaN,NaN,NaN,171.4,3.726087,179,3.891304,7.28,6.20,5.52,5.00,4.56,4.16,3.76,3.35,2.86,5.69,5.38,5.17,4.99,4.84,4.68,4.52,4.34,4.11,Amos Willingham,26128,686294
5210,Steven Wilson,CHW,1.6772,2.2339,0.0000,4.98717,40.00,0.00,0.0,0.6084,NaN,40.000,177.226,38.0698,23.3800,22.1652,7.29333,19.6206,1.1979,1.9872,37.9438,8.53736,4.41463,1.93388,1.641,0.214098,0.110709,0.103389,0.244636,1.442260,0.278820,0.722633,0.285320,NaN,5.26021,-0.230440,-0.217122,999.000000,NaN,NaN,NaN,NaN,NaN,NaN,125.1,3.127500,123,3.075000,7.89,6.71,5.96,5.40,4.91,4.47,4.05,3.60,3.06,6.11,5.79,5.57,5.39,5.22,5.06,4.90,4.71,4.47,Steven Wilson,20353,621051
5211,Evan Justice,COL,1.8032,2.3041,0.0000,5.13068,42.00,0.00,0.0,0.5577,NaN,42.000,191.868,42.5310,25.57

In [20]:
df = pd.DataFrame()
for proj_file in fg_types:
    try:
        df = pd.concat([df, pd.read_csv(f'data/{datetime.now().year}-{proj_file}-proj-h.csv', encoding="latin-1")])
        print(f'Found {proj_file}')
    except:
        print(f"Did not find file {proj_file}")
        pass

df.rename(columns={'PlayerId':'playerid', 'ï»¿Name':'Name'},inplace=True)
df = df.sort_values('playerid')

Did not find file zips
Did not find file zipsdc
Found steamer
Found fangraphsdc
Did not find file atc
Did not find file thebat
Did not find file thebatx


In [21]:
df['Primary_Pos'] = df.apply(lambda x: self.find_primary_pos(x['Pos']), axis=1)

NameError: name 'fu' is not defined